# 🌍 Exploratory Data Analysis - World Happiness Report (2015–2019)


## Pendahuluan

Dataset ini berisi laporan kebahagiaan dunia (World Happiness Report) dari tahun 2015 hingga 2019. 
Setiap baris mewakili satu negara pada satu tahun tertentu, dengan beberapa fitur yang berkontribusi pada skor kebahagiaan (Happiness Score).

**Tujuan EDA:**
- Membersihkan data (handling missing values & duplicates)
- Melakukan eksplorasi data deskriptif
- Menyiapkan data untuk dashboard interaktif Streamlit


1. Import dan Load Data

In [ ]:
# Import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Load semua file dari 2015–2019
data_path = "data"
years = range(2015, 2020)
dfs = []

for year in years:
    file_path = os.path.join(data_path, f"{year}.csv")
    df = pd.read_csv(file_path)
    df["Year"] = year
    dfs.append(df)

# Gabungkan semua tahun
df_all = pd.concat(dfs, ignore_index=True)

# Tampilkan info awal
print("Shape:", df_all.shape)
df_all.head()


2. Pemeriksaan Awal dan Cleaning

In [ ]:
# Info kolom
df_all.info()

# Cek missing value
df_all.isnull().sum()

# Cek duplikat
print("Jumlah duplikat:", df_all.duplicated().sum())

# Hapus duplikat
df_all = df_all.drop_duplicates()


3. Visualisasi Distribusi & Korelasi

In [ ]:
# Visualisasi distribusi skor kebahagiaan
plt.figure(figsize=(8,5))
sns.histplot(df_all['Happiness Score'], bins=30, kde=True)
plt.title("Distribusi Happiness Score")
plt.xlabel("Happiness Score")
plt.show()

# Korelasi antar fitur (jika kolom sesuai)
numeric_cols = df_all.select_dtypes(include=np.number)
plt.figure(figsize=(10, 8))
sns.heatmap(numeric_cols.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Korelasi Antar Variabel Numerik")
plt.show()


4. Simpan Versi Bersih

In [ ]:
# Simpan hasil bersih ke file baru
df_all.to_csv("world_happiness_clean.csv", index=False)